In [1]:
import csv
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd

#### [le monde]

In [2]:
url = 'https://www.lemonde.fr/actualite-en-continu/'

links = []

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

sections = soup.find_all('section')

for section in sections:
    if 'teaser' in section['class'] and section['class'][0] == 'teaser':
        links.append(section.find('a')['href'])

for index, link in enumerate(links):
    print(f'[{index}] {link}')

[0] https://www.lemonde.fr/societe/article/2024/10/23/nourrisson-enleve-dans-une-maternite-la-voiture-des-parents-retrouvee-a-charleroi-en-belgique_6358706_3224.html
[1] https://www.lemonde.fr/planete/article/2024/10/23/l-activiste-paul-watson-defenseur-des-baleines-maintenu-en-detention-au-groenland-jusqu-a-la-mi-novembre_6358704_3244.html
[2] https://www.lemonde.fr/international/live/2024/10/23/en-direct-presidentielle-americaine-kamala-harris-et-donald-trump-dans-les-swing-states-pour-consolider-leur-base-et-convaincre-les-indecis_6357232_3210.html
[3] https://www.lemonde.fr/international/article/2024/10/23/dans-le-detroit-de-taiwan-un-porte-avions-deploye-par-la-chine-au-lendemain-de-nouvelles-man-uvres_6358696_3210.html
[4] https://www.lemonde.fr/international/live/2024/10/23/en-direct-guerre-en-ukraine-kiev-appelle-tout-soldat-nord-coreen-sur-le-point-d-etre-mobilise-dans-le-pays-a-se-rendre_6356429_3210.html
[5] https://www.lemonde.fr/societe/article/2024/10/23/au-proces-des-vio

In [3]:
text=[]

for index, link in enumerate(links):
    article_response = requests.get(link,allow_redirects = False)
    article_soup = BeautifulSoup(article_response.text, 'html.parser')

    if article_soup.find('h1', class_='article__title') is None:
        pass

    else:
        title = article_soup.find('h1', class_='article__title')
        title_text = title.get_text(strip=True)
        text.append(title_text)

        desc = article_soup.find('p', class_='article__desc')
        desc_text = desc.get_text(strip=True)
        desc_text = re.sub(r'\s+', ' ', desc_text)  #공백이 여러 개 있으면 하나의 공백으로 바꿔주기
        text.append(desc_text)

        paragraphs=article_soup.find_all('p',class_='article__paragraph')
        for paragraph in paragraphs:
            paragraph_text=paragraph.get_text(strip=True)
            text.append(paragraph_text)

        print(f"[{index}] {link}")
        print(f"제목: {title_text}")
        print(f"설명: {desc_text}")
        print(f"본문: {paragraph_text}")

        print("\n" + "="*40 + "\n")

[0] https://www.lemonde.fr/societe/article/2024/10/23/nourrisson-enleve-dans-une-maternite-la-voiture-des-parents-retrouvee-a-charleroi-en-belgique_6358706_3224.html
제목: Nourrisson enlevé dans une maternité : la voiture des parents retrouvée à Charleroi en Belgique
설명: Santiago, âgé de dix-sept jours, était suivi comme grand prématuré dans un hôpital d’Aulnay-sous-Bois, près de Paris. Ses parents âgés d’une vingtaine d’années sont soupçonnés de l’avoir enlevé.
본문: 


[1] https://www.lemonde.fr/planete/article/2024/10/23/l-activiste-paul-watson-defenseur-des-baleines-maintenu-en-detention-au-groenland-jusqu-a-la-mi-novembre_6358704_3244.html
제목: L’activiste Paul Watson, défenseur des baleines, maintenu en détention au Groenland jusqu’à la mi-novembre
설명: La justice a décidé, mercredi, de prolonger la détention du fondateur de Sea Sheperd jusqu’au 13 novembre, pour la quatrième fois depuis l’arrestation en juillet du militant écologiste, menacé d’extradition au Japon.
본문: Quelques dizain

In [4]:
text

['Nourrisson enlevé dans une maternité\xa0: la voiture des parents retrouvée à Charleroi en Belgique',
 'Santiago, âgé de dix-sept jours, était suivi comme grand prématuré dans un hôpital d’Aulnay-sous-Bois, près de Paris. Ses parents âgés d’une vingtaine d’années sont soupçonnés de l’avoir enlevé.',
 'La piste d’une fuite en Belgique se confirme. La voiture du couple ayantenlevé son nouveau-né grand prématuré dans une maternitéde la région parisienne a été retrouvée à Charleroi, mercredi 23\xa0octobre, dans la matinée.',
 'Le véhicule de marque Audi, dont la photo et la plaque d’immatriculation française figuraient dans l’avis de recherche lancé mardi soir en Belgique, a été découvert«\xa0vers 9\xa0heures\xa0», a souligné un porte-parole de la police de Charleroi, sans autres précisions. Ni le parquet de Mons, chargé des investigations en Belgique, ni celui de Bobigny en France n’ont commenté l’information.',
 'Santiago, âgé de dix-sept jours, était suivi comme grand prématuré dans un

In [5]:
text[0]

'Nourrisson enlevé dans une maternité\xa0: la voiture des parents retrouvée à Charleroi en Belgique'

In [6]:
text[2]

'La piste d’une fuite en Belgique se confirme. La voiture du couple ayantenlevé son nouveau-né grand prématuré dans une maternitéde la région parisienne a été retrouvée à Charleroi, mercredi 23\xa0octobre, dans la matinée.'

In [7]:
file_name=f'./lemonde_{datetime.today().strftime("%Y%m%d")}.txt'

with open(file_name,'w+') as file:
    file.write('\n'.join(text))

#### [rfi]

In [8]:
url2 = 'https://www.rfi.fr/en/france/'

links2 = []

headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

response = requests.get(url2, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

divs=soup.find_all('div')

for div in divs:
    print(div)
#     if 'article__title' in div['class']:
#         links.append(div.find('a')['href'])

# for index, link in enumerate(links):
#     print(f'[{index}] {link}')

<div class="o-header__inner"><div class="o-header__inner__background brand"><a class="o-header__home-link a-home-link" href="/"><span class="a-brand-logo a-brand-logo--rfi"></span></a></div></div>
<div class="o-header__inner__background brand"><a class="o-header__home-link a-home-link" href="/"><span class="a-brand-logo a-brand-logo--rfi"></span></a></div>
<div class="err-num">404</div>
<div class="description"><p>Le contenu auquel vous tentez d'accÃ©der n'existe pas ou n'est plus disponible.</p></div>


#### [Doctissimo]

In [9]:
url = 'https://www.doctissimo.fr/'

links = []

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

spans = soup.find_all('span', class_='h5 card__body__title')

for span in spans:
    #print(span)
    atag=span.find('a')
    links.append(atag['href'])

for index, link in enumerate(links):
    print(f'[{index}] {link}')

[0] https://www.doctissimo.fr/nutrition/5-aliments-riches-en-calories-qui-sont-excellents-pour-votre-sante-selon-une-nutritionniste/8cafe0_ar.html
[1] https://www.doctissimo.fr/sante/le-depistage-du-cancer-du-poumon-est-il-realisable-en-vie-reelle-oui-selon-un-pneumo-oncologue/78f278_ar.html
[2] https://www.doctissimo.fr/famille/relations-familiales/les-enfants-sont-rois-que-dire-a-un-proche-qui-surexpose-ses-enfants-en-ligne-les-conseils-dune-psy/968202_ar.html
[3] https://www.doctissimo.fr/psychologie/relation-amoureuse-et-psychologie/la-regle-des-3c-essentielle-pour-quun-homme-tombe-amoureux-selon-un-coach-en-relations/e7df3c_ar.html
[4] https://www.doctissimo.fr/nutrition/thon-ou-saumon-quel-poisson-choisir-pour-votre-sante-lavis-dune-dieteticienne/27e150_ar.html
[5] https://www.doctissimo.fr/sante/un-americain-se-reveille-pendant-le-prelevement-de-ses-organes-en-france-le-protocole-pour-eviter-cela/0d29c3_ar.html
[6] https://www.doctissimo.fr/sante/maladies/cancer/cancer-du-sein-p

In [10]:
text=[]

for index, link in enumerate(links):
    print(f"[{index}] {link}")

    article_response = requests.get(link,allow_redirects = False)
    article_soup = BeautifulSoup(article_response.text, 'html.parser')

    title = article_soup.find('h1')
    title_text = title.get_text(strip=True)
    #print(title_text)
    text.append(title_text)

    desc = article_soup.find('div', class_='doc-my-4')
    if desc==None: pass
    else: 
        desc_text = desc.get_text(strip=True)
        #print(desc_text)
        desc_text = re.sub(r'\s+', ' ', desc_text)
        text.append(desc_text)

    contents=article_soup.find('div', class_='doc-content doc-px-2 doc-px-md-0')
    if contents==None: pass
    else:
        p_tags=contents.find_all('p',class_=False)

        for p in p_tags:
            p_text=p.get_text(strip=True)
            #print(p_text)
            text.append(p_text)

    print(f"제목: {title_text}")
    print(f"설명: {desc_text}")
    print(f"본문: {p_text}")

    print("\n" + "="*40 + "\n")

[0] https://www.doctissimo.fr/nutrition/5-aliments-riches-en-calories-qui-sont-excellents-pour-votre-sante-selon-une-nutritionniste/8cafe0_ar.html
제목: 5 aliments riches en calories qui sont excellents pour votre santé, selon une nutritionniste
설명: On pense souvent que tous les aliments riches en calories sont néfastes pour la santé. Une grossière erreur, selon notre experte Alexandra Murcier, diététicienne-nutritionniste.
본문: Voici donc les 5 aliments caloriques à privilégier pour faire du bien à votre santé.


[1] https://www.doctissimo.fr/sante/le-depistage-du-cancer-du-poumon-est-il-realisable-en-vie-reelle-oui-selon-un-pneumo-oncologue/78f278_ar.html
제목: Le dépistage du cancer du poumon est-il réalisable en vie réelle ? Oui, selon un pneumo-oncologue
설명: Le dépistage du cancer du poumon sera-t-il instauré un jour en France ? Et si oui, est-il réalisable en conditions réelles ? Pour répondre à ces questions, Doctissimo a interrogé le Dr Olivier Leleu, pneumo-oncologue et auteur d’un

In [11]:
len(text)

1193

In [12]:
file_name=f'./doctissimo_{datetime.today().strftime("%Y%m%d")}.txt'

with open(file_name,'w+') as file:
    file.write('\n'.join(text))

#### [텍스트 전처리-lemonde]

In [13]:
with open('lemonde_20241023.txt','r') as f:
    lemonde=f.readlines()
    for l in lemonde:
        l=l.lower()                             #소문자로 바꾸기
        l=re.sub(r'[^a-zA-ZÀ-ÿ0-9\s]', '', l)   #특수문자 제거
        print(l)

nourrisson enlevé dans une maternité  la voiture des parents retrouvée à charleroi en belgique

santiago âgé de dixsept jours était suivi comme grand prématuré dans un hôpital daulnaysousbois près de paris ses parents âgés dune vingtaine dannées sont soupçonnés de lavoir enlevé

la piste dune fuite en belgique se confirme la voiture du couple ayantenlevé son nouveauné grand prématuré dans une maternitéde la région parisienne a été retrouvée à charleroi mercredi 23 octobre dans la matinée

le véhicule de marque audi dont la photo et la plaque dimmatriculation française figuraient dans lavis de recherche lancé mardi soir en belgique a été découvert vers 9 heures  a souligné un porteparole de la police de charleroi sans autres précisions ni le parquet de mons chargé des investigations en belgique ni celui de bobigny en france nont commenté linformation

santiago âgé de dixsept jours était suivi comme grand prématuré dans un hôpital daulnaysousbois seinesaintdenis près de paris il est acti

In [14]:
#불용어 리스트
stopword='stopwords-fr.txt'

with open(stopword, 'r', encoding='utf-8') as f:
    stop_words = [line.strip() for line in f]
    print(stop_words)

['a', 'abord', 'absolument', 'afin', 'ah', 'ai', 'aie', 'aient', 'aies', 'ailleurs', 'ainsi', 'ait', 'allaient', 'allo', 'allons', 'allô', 'alors', 'anterieur', 'anterieure', 'anterieures', 'apres', 'après', 'as', 'assez', 'attendu', 'au', 'aucun', 'aucune', 'aucuns', 'aujourd', "aujourd'hui", 'aupres', 'auquel', 'aura', 'aurai', 'auraient', 'aurais', 'aurait', 'auras', 'aurez', 'auriez', 'aurions', 'aurons', 'auront', 'aussi', 'autant', 'autre', 'autrefois', 'autrement', 'autres', 'autrui', 'aux', 'auxquelles', 'auxquels', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avez', 'aviez', 'avions', 'avoir', 'avons', 'ayant', 'ayez', 'ayons', 'b', 'bah', 'bas', 'basee', 'bat', 'beau', 'beaucoup', 'bien', 'bigre', 'bon', 'boum', 'bravo', 'brrr', 'c', 'car', 'ce', 'ceci', 'cela', 'celle', 'celle-ci', 'celle-là', 'celles', 'celles-ci', 'celles-là', 'celui', 'celui-ci', 'celui-là', 'celà', 'cent', 'cependant', 'certain', 'certaine', 'certaines', 'certains', 'certes', 'ces', 'cet', 'cette', 'ce

In [15]:
len(stop_words)

691

In [16]:
!pip install transformers


In [17]:
!pip install sentencepiece

In [18]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification

c:\Users\MSI\anaconda3\envs\WEB_AI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
tokenizer = CamembertTokenizer.from_pretrained('camembert/camembert-base')
model = CamembertForSequenceClassification.from_pretrained('camembert/camembert-base')

c:\Users\MSI\anaconda3\envs\WEB_AI\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MSI\.cache\huggingface\hub\models--camembert--camembert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert/

In [ ]:
train_tokens = [[token for token in tokenizer.morphs(text) if token not in stop_words] for text in trainDF['SentimentText']]
